#NOTEBOOK 6/6: SQL DASHBOARD QUERIES

##QUERY 1: OVERALL METRICS

In [0]:
%sql
SELECT 
    COUNT(*) as total_encounters,
    SUM(readmitted_30days) as total_readmissions,
    ROUND(AVG(readmitted_30days) * 100, 2) as readmission_rate_pct,
    ROUND(AVG(time_in_hospital), 2) as avg_length_of_stay
FROM healthcare_analytics.silver.silver_events;

##QUERY 2: TOP 10 WORST PERFORMING SPECIALTIES

In [0]:
%sql
SELECT * 
FROM healthcare_analytics.gold.readmission_by_specialty
ORDER BY readmission_rate_pct DESC
LIMIT 10;

##QUERY 3: TOP 10 BEST PERFORMING SPECIALTIES

In [0]:
%sql
SELECT * 
FROM healthcare_analytics.gold.readmission_by_specialty
ORDER BY readmission_rate_pct ASC
LIMIT 10;

##QUERY 4: HIGH-RISK PATIENTS

In [0]:
%sql
SELECT 
    patient_nbr,
    total_visits,
    total_readmissions,
    patient_risk_tier,
    max_medications,
    max_diagnoses
FROM healthcare_analytics.gold.patient_summary
WHERE patient_risk_tier IN ('Critical', 'High')
ORDER BY total_readmissions DESC
LIMIT 100;

##QUERY 5: READMISSION BY AGE GROUP

In [0]:
%sql

SELECT * 
FROM healthcare_analytics.gold.age_group_analysis
ORDER BY readmission_rate_pct DESC;

##QUERY 6: MEDICATION IMPACT

In [0]:
%sql
SELECT * 
FROM healthcare_analytics.gold.medication_impact
ORDER BY readmission_rate_pct DESC;

##QUERY 7: EMERGENCY VS SCHEDULED ADMISSIONS

In [0]:
%sql
SELECT 
    CASE 
        WHEN is_emergency = 1 THEN 'Emergency'
        ELSE 'Scheduled'
    END as admission_type,
    COUNT(*) as total_encounters,
    SUM(readmitted_30days) as readmissions,
    ROUND(AVG(readmitted_30days) * 100, 2) as readmission_rate_pct
FROM healthcare_analytics.silver.silver_events
GROUP BY is_emergency;

##QUERY 8: LENGTH OF STAY ANALYSIS

In [0]:
%sql
SELECT 
    time_in_hospital_category,
    COUNT(*) as patient_count,
    ROUND(AVG(readmitted_30days) * 100, 2) as readmission_rate_pct
FROM healthcare_analytics.silver.silver_events
GROUP BY time_in_hospital_category
ORDER BY readmission_rate_pct DESC;

##QUERY 9: PATIENT RISK DISTRIBUTION

In [0]:
%sql
SELECT 
    patient_risk_tier,
    COUNT(*) as patient_count
FROM healthcare_analytics.gold.patient_summary
GROUP BY patient_risk_tier
ORDER BY patient_count DESC;

##QUERY 10: COST IMPACT CALCULATION

In [0]:
%sql
SELECT 
    SUM(readmitted_30days) as total_readmissions,
    SUM(readmitted_30days) * 15000 as total_cost_dollars,
    ROUND(SUM(readmitted_30days) * 15000 * 0.20, 0) as potential_savings_20pct
FROM healthcare_analytics.silver.silver_events;

##QUERY 11: FUTURE READMISSION RISK - TOP 100 PATIENTS


In [0]:

%sql
SELECT 
    patient_nbr,
    ROUND(readmission_risk_score * 100, 2) as risk_percentage,
    risk_category,
    age_group,
    num_medications,
    number_diagnoses,
    medical_specialty,
    recommended_action
FROM healthcare_analytics.gold.future_readmission_risk
WHERE actual_readmitted = 0  -- Patients who haven't been readmitted yet
ORDER BY readmission_risk_score DESC
LIMIT 100;

##QUERY 12: RISK SCORE DISTRIBUTION

In [0]:


%sql
SELECT 
    risk_category,
    COUNT(*) as patient_count,
    ROUND(AVG(readmission_risk_score) * 100, 2) as avg_risk_pct,
    SUM(CASE WHEN actual_readmitted = 1 THEN 1 ELSE 0 END) as actual_readmissions
FROM healthcare_analytics.gold.future_readmission_risk
GROUP BY risk_category
ORDER BY avg_risk_pct DESC;